In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import CombinedData
from PIL import Image
from matplotlib import pyplot as plt

hasy_train = CombinedData('HASY')
hasy_test = CombinedData('HASY', train=False)

print("Train data length: {0}".format(len(hasy_train.data)))
print("Test data length: {0}".format(len(hasy_test.data)))
print("Img Shape: {0}".format(hasy_train.data[0].shape))
print("Number of Labels: {0}".format(hasy_train.no_labels))

100%|██████████| 10000/10000 [00:00<00:00, 342372.60it/s]

Train data length: 65878
Test data length: 10832
Img Shape: torch.Size([1, 28, 28])
Number of Labels: 15


In [2]:
from torchvision import models
from torch.nn import Conv2d

train_loader = DataLoader(hasy_train, batch_size=16, shuffle=True)
test_loader = DataLoader(hasy_test, batch_size=16, shuffle=False)

def calc_accuracy(model):
    accuracies = []
    for idx, [x_test, y_test] in enumerate(test_loader):
        test_pred = model(x_test)
        accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
        accuracies.append(accuracy)
    return np.mean(accuracies) 

def print_stats(lr, beta1, beta2, decay, accuracy):
    print("Learning rate: {0}".format(lr))
    print("Beta 1: {0}".format(beta1))
    print("Beta 2: {0}".format(beta2))
    print("Weight decay: {0}".format(decay))
    print("Accuracy: {0}".format(accuracy))

learning_rates = [0.01, 0.001, 0.0001]
betas1 = [0.8, 0.85, 0.9, 0.95]
betas2 = [0.9, 0.925, 0.95, 0.99]
weight_decays = [0, 0.01, 0.001, 0.0001]

best_lr = 0
best_beta1 = 0
best_beta2 = 0
best_decay = 0
best_accuracy = 0

for lr in learning_rates:
    for beta1 in betas1:
        for beta2 in betas2:
            for decay in weight_decays:
                model = models.alexnet(num_classes=15)
                model.features[0] = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=decay)
                criterion = nn.CrossEntropyLoss()
                for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
                    optimizer.zero_grad()
                    train_pred = model(x_train)
                    loss = criterion(train_pred, y_train)
                    loss.backward()
                    optimizer.step()
                accuracy = calc_accuracy(model)
                print_stats(lr, beta1, beta2, decay, accuracy)
                if accuracy > best_accuracy:
                    best_lr = lr
                    best_beta1 = beta1
                    best_beta2 = beta2
                    best_decay = decay
                    best_accuracy = accuracy
                    
print("Best hyperparameters:")
print_stats(best_lr, best_beta1, best_beta2, best_decay, best_accuracy)
                

 46%|████▌     | 1901/4118 [51:23<1:32:06,  2.49s/it]

KeyboardInterrupt: 